For this assignment you will be implementing your own verision of the perceptron algorithm.  Similar to the Naive Bayes assignment your Percptron class should be built to be compatible with the sklearn framework.  You can use the sklearn library and other python libraries to help with your implementation but you must implement the actual percpetron algorithm using your own code.

In [ ]:
# import required modules
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, balanced_accuracy_score # balanced_accuracy_score with adjusted=True is Informedness
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
import numpy as np

In [ ]:
#TODO: put your perceptron implemenation here

#initial imports that you may find useful
from sklearn.base import BaseEstimator, ClassifierMixin
import itertools 

class Perceptron(BaseEstimator, ClassifierMixin):
    def __init__(self, alpha=1, epochs=10):
        self.alpha = alpha
        self.epochs = epochs
             
    def fit(self, X, y):

        def ApplyandReweight(alpha, weights, X, y):

            y_hat = np.sum(np.multiply(weights,X))
            print(f"y: {y} y_hat: {y_hat} = {weights} * {X}")
            if (y_hat >= 1 and y==1) or (y_hat < 1 and y == -1):
                new_weights = weights
                print(f"Good prediction. Return same weights")
            else:
                new_weights = weights + (alpha * (y - y_hat) * X)
                print(f"Bad prediction. new_weights = weights + (alpha * (y - y_hat) * X): {weights} + ({alpha} * ({y_hat} - {y}) * {X})")
            return new_weights
        
        # STEP 0: Convert dataframe to numpy array
        # X = X.to_numpy()
        # y = y.to_numpy()

        # STEP 1: Initialize the weights of the parameters
        X_rows, X_columns = X.shape
        # print(f"X shape: {X_rows}, {X_columns}")
        weights = np.random.rand(X_columns) # Create an array with the number of parameters as X
        print(f"Initial random weights: {weights}")

        # STEP 2: Apply the weights to the Nth instance
        epoch = 1
        while epoch <= self.epochs:
            for i, row in enumerate(y):                    # Iterate through both X and y together. i is the counter, row is X[i]
                print(f"\nRow[{i}]")
                weights = ApplyandReweight(self.alpha, weights, X[i], row)
        
        return self
    
    def predict(self, X):

        return np.array(y_predicted) # For a long time I was returning self



Split the diabetes data into and 80/20 train/test split.


In [ ]:
#TODO: split the diabetes data into training/test data

df = pd.read_csv('Datasets for Assignment 4/diabetes.csv')
X = df.drop(columns='class')
y = df['class']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=42)

Using the diabetes training data with k-fold cross-validation (you choose the value for k) plot your perceptron's cross-validation accuracy and MCC as a function of the number of training epochs.  Use a learning rate of 0.10.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
#TODO: k-fold cross validation with a learning rate of 0.10

pipe = make_pipeline(MinMaxScaler(), Perceptron(alpha=0.1, epochs=3))
pipe.fit(X_train, y_train)


Choose a normalization/standardization strategy to apply to the diabetes training data. Using k-fold cross validation and plot the cross validation accuracy and MCC as a function of the number of training epochs.  Repeat for learning rates of 0.90, 0.50, 0.10, 0.01, and 0.0001.

In [ ]:
#TODO: k-fold cross validation on normalized/standardized data

How does the normalization/standardization affect the accuracy and MCC scores and number of training epochs?

Answer in this text box

On the scaled data, for each learning rate, how many training epochs are needed to acheive maximize the accuracy and MCC scores?

Answer in this text box

Is there a correlation between learning rates and the number of training epochs needed to acheive the optimal results?  If so, what is the correlation?  If not, why not?

Answer in this text box